In [6]:
import numpy as np

import pysqkit as sq

from IPython.display import display, Latex

In [2]:
transmon = sq.qubits.SimpleTransmon(
    label='transmon', 
    max_freq = 4.5, 
    anharm = -0.3, 
    dim_hilbert = 3,
    diel_loss_tan=0.75*1e-6
)

In [3]:
fluxonium = sq.qubits.Fluxonium(
    label='fluxonium',
    joseph_energy=8.0, 
    charge_energy=0.973, 
    induct_energy=0.457,
    diel_loss_tan=7.1*1e-6
)
fluxonium.diagonalize_basis(5)

In [4]:
system = transmon.couple_to(fluxonium, coupling=sq.couplers.capacitive_coupling, strength=0.2)

system['fluxonium'].add_drive(
    sq.drives.microwave_drive,
    label='cz_drive',
    pulse_shape=sq.drives.pulse_shapes.gaussian_top
)

In [10]:
for qubit in system:
    relax_rate, exc_rate = qubit.loss_rates(0, 1)
    relax_time = 1/(2*np.pi*relax_rate)
    display(Latex(r'$T_{{1}}^{{{}}} $ = {:.3f} $\mu s$'.format(qubit.label, relax_time*1e-3)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [45]:
times = np.linspace(0, 60, 10000)
system['fluxonium'].drives['cz_drive'].set_params(phase=0, time=times, rise_time=15)

In [50]:
drive_params = dict(
    cz_drive=dict(amp = 0.03634, freq = 5.19)
)

hamil = system.hamiltonian(as_qobj=True)
drive_hamils = [drive.hamiltonian(as_qobj=True) for qubit in system for drive in qubit.drives.values() if qubit.is_driven]
pulses = [drive.eval_pulse(**drive_params[label]) for qubit in system for label, drive in qubit.drives.items()]
collapse_ops = [op for qubit in system for op in qubit.collapse_ops(as_qobj=True)]